<a href="https://colab.research.google.com/github/RKayyy/102116008-SESS_LE1/blob/main/Demo_NB_102116008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **DEMO NOTEBOOK**

Speech Recognition Lab Evaluation 1

Name: Reitika Kumar
Roll No.: 102116008
Date: 12/9/24

Submitted To: Dr. Raghav B. Venkataramaiyer



In [3]:
!pip install tensorflow librosa


### **Loading the Model**

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense

def create_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

input_shape = (13, 1)
num_classes = 38

model = create_model(input_shape, num_classes)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### **Loading Weights**

In [5]:
model.load_weights('/content/model_weights.weights.h5')
print("Model weights loaded successfully.")


Model weights loaded successfully.


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### **Loading the custom dataset**

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!cp /content/drive/MyDrive/speech_recog_lab_eval/custom_data.zip /content/


In [8]:
!unzip /content/your_dataset.zip -d /content/custom_data/


unzip:  cannot find or open /content/your_dataset.zip, /content/your_dataset.zip.zip or /content/your_dataset.zip.ZIP.


In [9]:
dataset_path = '/content/drive/MyDrive/speech_recog_lab_eval/custom_data'


### **Testing the model with a given .wav audio file**

In [10]:
import librosa
import numpy as np

def extract_features(file_path, sr=16000):
    y, sr = librosa.load(file_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

sample_file_path = '/content/0a1c3j4e_nohash_4.wav' #uploaded file from 'eight' directory

test_features = extract_features(sample_file_path)

test_features_cnn = test_features.reshape(1, test_features.shape[0], 1)


In [11]:
from sklearn.preprocessing import LabelEncoder

commands = ['backward', 'bed','bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']  # Add all 35 commands here

le = LabelEncoder()
le.fit(commands)

predictions = model.predict(test_features_cnn)

predicted_class_index = np.argmax(predictions, axis=1)[0]

predicted_command = le.inverse_transform([predicted_class_index])

print(f'Predicted Command: {predicted_command[0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Predicted Command: eight


### **Checksum**

In [12]:
import hashlib

def get_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

model_checksum = get_md5('/content/model_weights.weights.h5')
dataset_checksum = get_md5('/content/custom_data.zip')

print(f'Model Checksum: {model_checksum}')
print(f'Dataset Checksum: {dataset_checksum}')


Model Checksum: 0a818cb48df623aed6940a461e140a46
Dataset Checksum: 854f8871298e7dd477006309597e21e2
